# Pipe sizing is more than a speed limit
## Client: INTERNAL
---
## Project: Blog article
## Calc: CALC-001, line sizing
---
## Authentication
> Stamp, Permit
---
## Revision History
|Revision | Date | Description | By | Reviewer|
| :-------| :----|:------------|:---|:--------|
|       | | DRAFT | KCD |  |
|         |          |            |     |    |

---

## Abstract

## Introduction
Sizing lines is a routine task in process engineering. University courses may not touch on this, perhaps becauase it seems so simple. I struggled with line sizing rationale when I was a junior engineer, and it did not help that I tended to research things rather than just asking a senior mentor. Perhaps I did not want to seem foolish for asking such a simple question.

We will limit the discussion to incompressible fluid, liquids or flowing gases that have a small pressure drop. We will exclude vapours in PSV applicaiton, because these have different criteria. We will also exclude liquids on the suction side of piston pumps, because this also haas a different sizing criteria.

The example that we will consider is a normal (material balance) flow rate of 50,000 kg/h of propane (density 500 kg/m3).

## Design margin on flow rate
Take some care with design margin on the flow rate. If the design flow rate of propane today is 50,000 kg/h, what might it be in 20 years after the plant has been debottlenecked? Apply a reasonable design margin above the flow rate that is produced from a material balance. It might be feasible for the plant to be designed to handle 30% additional flow. We will consider a design margin of 30% in our example.



In [1]:
# import standard
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy as sp
from scipy import interpolate
from scipy.interpolate import interp1d
from tabulate import tabulate



## Criteria: maximum velocity
I used a term without explaining it: criteria. This defines what the pipe flow will achieve with our chosen size. The simplest criteria is to specify a maximum velocity.
* API 945 recommends a maximum velocity of 1.8 m/s for rich amine.
* API 14E recommends a maximum velocity related to the density of the fluid, $v_{max} = C/\sqrt{\rho}$ where C is a parameter dependent on the service (often 120), the density is in $kg/m^3$ and the maximum flow is in m/s. The specification of an erosional C factor is often the only velocity consideration that an engineer may give to the problem.
* for fluids that are corrosion inhibited, NOROSK recommends limiting the wall shear stress to 40 N/m2, which implies $v_{max} = \sqrt{80/(f\,\rho)}\,\,(m/s)$ where $f$ is the fanning friction factor.

The math is straight forward.
$$
v_{max} = \frac{\dot{m}}{\rho\,(\pi/4)\,D^2} 
$$
$$
D^2  = \frac{\dot{m}}{\rho\,(\pi/4)\,v_{max}} 
$$
$$
D  = \sqrt{\frac{4\,\dot{m}}{\rho\,\pi\,v_{max}} }
$$

The erosional C factor (in SI) is related to the C factor from API 14E (in US Customary units). 
* give a few examples of C.


In [2]:
# example 50,000 kg/h liquid propane, density 500 kg/m3
# use a C factor of 120
C = 120.0
rho = 500.0
designMargin = 1.3
mdot = designMargin * 50000.0 / 3600.0 # kg/s
v_max = C/math.sqrt(rho)
D = math.sqrt(4*mdot/(rho*math.pi*v_max))
mdot, v_max, D

(18.055555555555557, 5.366563145999495, 0.09256085617953327)

The maximum velocity is 5.4 m/s, and the minimum diameter is 93 mm. This suggests that NPS 4 (102 mm ID) is adequate.

In [3]:
# NOROSK criteria
# vmax = sqrt(80 / (rho f_moody))
rho = 1000.0
ffanning = 0.015/4.0
vmaxNOROSK = math.sqrt(80/(rho*ffanning))
Cfactor = math.sqrt(80/ffanning)
vmaxNOROSK, Cfactor

(4.618802153517006, 146.0593486680443)

## Pressure drop per unit pipe length
Erosional velocities determine the upper limit for the velocity to prevent long term damage to the piping. Another criteria is the pressure drop per unit length. This is more related to the economics of piping, the tradeoff between the capital cost of larger piping, and the cost of lost pressure energy.  The pressure drop is also important to ensure adequate supply pressure where our fluid is delivered to various consumers in the plant. This ensures that a sufficient pressure drop will exist across the supply control valve to permit adequate flow control for the fluid.

The choice for an appropriate pressure drop per 100 m of piping requires some artisic license. A utility such as steam will have a common header and branch laterals: the pressure drop criteria for these two services are usually different. I have not seen very many assessments of line size to minimize life cycle cost.

Note to self. Is there an opportunity to produce a criteria for line sizing based on life cycle cost?

NOROSK provides the following guidelines for gas piping, and these values are in-line with guidelines from other orgnizations.

|Operating pressure, kPag | Pressure drop kPa per 100 m|
| :-----------------------|:--------|
|  0 - 3500               | 0.1 - 11 |
|  3500 - 13,800          |  11 - 27  |

For example, consider a boiler that produces 350 kPag steam, and we need a minimum supply pressure of 330 kPag for all consumers. A large steam header travels the distance of the plant (say 200 m long), and then branch lines must travel typically 20 m to steam consumers. A design criteria of 5 kPa per 100 m for the main steam header, and 10 kPa per 100 m for the smaller steam laterals would be a reasonable sizing criteria.

For this criteria, an old reference from Perry's Handbook (5th edition) is useful.  Swamee and Jain provided two correlations for the Moody friction factor.  The first correlation is an explicit form in terms of the pipe Reynolds number and relative roughness. The second correlation relates the friction factor to the frictional head loss per unit length of pipe, volumetric flow rate, fluid properties and pipe roughness: it is independant of pipe diameter.

$$
\frac{D^5 g S_f}{q^2} = \frac{1}{8} \left[
\left(\frac{\epsilon^5 g S_f}{q^2}\right)^{1/4} 
+ \left(\frac{\nu^5}{q^3 g S_f}\right)^{1/5}
\right]^{1/5}
$$
where
* ε =  absolute roughness
* Sf  is the head loss of fluid per unit pipe length 
* Sf = Δp/(ρgL)
* ν = kinematic viscosity 
* ν = μ/ρ
* D = pipe diameter
* q = volumetric flow rate.

In [7]:
# same example, but lets specify a pressure drop of 50 kPa per 100 m
# use roughness 0.05 mm for commercial steel
dp = 50.0
L = 100.0
g = 9.81
mu = 0.1e-3 # Pa.s
rho = 500.0
nu = mu/rho

eps = 0.05 / 1000.0 # m


Sf = (dp*1000)/(rho*g*L)

designMargin = 1.3
mdot = designMargin * 50000.0 / 3600.0 # kg/s

#q = mdot/rho
q = mdot /rho

term1 = (eps**5)*g*Sf/(q**2)
term2 = (nu**5)/((q**3)*g*Sf)
RHS = (1.0/8.0)*(term1**(1./4.) + term2**(1./5.))**(1./5.)
D5 = RHS*(q**2)/(g*Sf)
D = D5**(1./5.)
D

0.11411672564912193

But we need 114 mm ID to satisfy the pressure drop criteria. The 4" line is too small. A NPS 6 line has ID 154 mm, and is more than adequate. It is left as an exercise for the reader to confirm the required line size with no additional flow margin (this should be 103 mm ID): the 4" line would work but would provide no flow margin.

## Afterthoughts

Be prudent with the line size. In most cases we should use the next larger line size. But if there is limited space for piping, then it may be better to use the smaller line and accept the reduced limit on the flow rate.

A consistant methodology for line sizing is an excellent engineering tool. This could be implemented in a spreadsheet or a Python application.

In [5]:
# NOROSK criteria
# vmax = sqrt(80 / (rho f_moody))
rho = 1000.0
ffanning = 0.015/4.0
vmaxNOROSK = math.sqrt(80/(rho*ffanning))
Cfactor = math.sqrt(80/ffanning)
vmaxNOROSK, Cfactor

(4.618802153517006, 146.0593486680443)

## References
* API 945
* API 14E
* NOROSK Standard P-001, rev 3 (1997).
* Swamee and Jain line sizing formula, Perrys handbook, 5th edition. Equation 5-65.
